## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under XGBoost directory that was uploaded in edX as a part of "Notebooks for weeks 7 & 8". 

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

### Loading Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [3]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [8]:
def calc_stats(margin_scores):
    #
    # YOUR CODE HERE
    #
    m = np.mean(margin_scores, 1)
    s = np.std(margin_scores, 1)
    return np.around(m - 3*s, 2), np.around(m + 3*s, 2)


In [9]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [10]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [11]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [12]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [13]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


### Calculating predictions

Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [20]:
def predict(min_scr, max_scr):
    #
    # YOUR CODE HERE
    #
    return  (max_scr > 0).astype(int) - (min_scr < 0).astype(int)


In [21]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])


In [22]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [23]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

**Procedure**

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
**Note**: You can experiment by changing **n_bootstraps**, but it takes about 200 iterations to get consistent values.

In [66]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
                   num_round=100, plst=xgboost_plst()):
    #
    # YOUR CODE HERE
    #
    margin_score = np.empty((y_test.shape[0], n_bootstrap))
    res = []
    for j in range(n_bootstrap):
        indices = random.choices([i for i in range(y_train.shape[0])], k=bootstrap_size)
        dtrain = xgb.DMatrix(X_train[indices], label=y_train[indices])
        dtest = xgb.DMatrix(X_test, label=y_test)
        evallist = [(dtest, 'eval'), (dtrain, 'train')]
        bst = xgb.train(plst, dtrain, num_round, evallist)
        pred = bst.predict(dtest, True)
        pred /= np.max(pred) - np.min(pred)
        margin_score[:, j] = np.around(pred, 2)
    return calc_stats(np.sort(margin_score)[:, int(np.ceil(n_bootstrap*minR)):int(np.floor(n_bootstrap*maxR))])       

In [67]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the interval(max_scr-min_scr) is not more than twice the average length of the interval

In [68]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[0]	eval-logloss:0.629825	train-logloss:0.555815
[1]	eval-logloss:0.60729	train-logloss:0.479572
[2]	eval-logloss:0.634136	train-logloss:0.429916
[3]	eval-logloss:0.586371	train-logloss:0.396712
[4]	eval-logloss:0.551655	train-logloss:0.372867
[5]	eval-logloss:0.518691	train-logloss:0.352113
[6]	eval-logloss:0.512676	train-logloss:0.337514
[7]	eval-logloss:0.524408	train-logloss:0.325979
[8]	eval-logloss:0.554189	train-logloss:0.314133
[9]	eval-logloss:0.579209	train-logloss:0.305317
[10]	eval-logloss:0.519926	train-logloss:0.296446
[11]	eval-logloss:0.519868	train-logloss:0.289246
[12]	eval-logloss:0.508521	train-logloss:0.282154
[13]	eval-logloss:0.491696	train-logloss:0.275185
[14]	eval-logloss:0.465639	train-logloss:0.271482
[15]	eval-logloss:0.474886	train-logloss:0.266232
[16]	eval-logloss:0.456896	train-logloss:0.260147
[17]	eval-logloss:0.449606	train-logloss:0.257469
[18]	eval-logloss:0.474316	train-logloss:0.253079
[19]	eval-logloss:0.46569	train-logloss:0.247734
[20]	eval-lo

[65]	eval-logloss:0.456335	train-logloss:0.128277
[66]	eval-logloss:0.471924	train-logloss:0.126466
[67]	eval-logloss:0.468795	train-logloss:0.124883
[68]	eval-logloss:0.480226	train-logloss:0.12296
[69]	eval-logloss:0.478437	train-logloss:0.121486
[70]	eval-logloss:0.489917	train-logloss:0.121005
[71]	eval-logloss:0.487754	train-logloss:0.120449
[72]	eval-logloss:0.488075	train-logloss:0.119693
[73]	eval-logloss:0.475808	train-logloss:0.118708
[74]	eval-logloss:0.472116	train-logloss:0.117294
[75]	eval-logloss:0.468627	train-logloss:0.115396
[76]	eval-logloss:0.468585	train-logloss:0.113807
[77]	eval-logloss:0.470924	train-logloss:0.112201
[78]	eval-logloss:0.482006	train-logloss:0.110948
[79]	eval-logloss:0.514501	train-logloss:0.110121
[80]	eval-logloss:0.490553	train-logloss:0.109247
[81]	eval-logloss:0.493589	train-logloss:0.108258
[82]	eval-logloss:0.522151	train-logloss:0.107638
[83]	eval-logloss:0.525479	train-logloss:0.106021
[84]	eval-logloss:0.538236	train-logloss:0.104705
[

[30]	eval-logloss:0.416938	train-logloss:0.219723
[31]	eval-logloss:0.418703	train-logloss:0.215113
[32]	eval-logloss:0.399807	train-logloss:0.212407
[33]	eval-logloss:0.400443	train-logloss:0.209436
[34]	eval-logloss:0.434194	train-logloss:0.205899
[35]	eval-logloss:0.424554	train-logloss:0.204393
[36]	eval-logloss:0.430437	train-logloss:0.202931
[37]	eval-logloss:0.445485	train-logloss:0.200327
[38]	eval-logloss:0.432221	train-logloss:0.199115
[39]	eval-logloss:0.427896	train-logloss:0.198108
[40]	eval-logloss:0.41714	train-logloss:0.194396
[41]	eval-logloss:0.442852	train-logloss:0.193199
[42]	eval-logloss:0.439689	train-logloss:0.192068
[43]	eval-logloss:0.436426	train-logloss:0.190023
[44]	eval-logloss:0.424121	train-logloss:0.187418
[45]	eval-logloss:0.424347	train-logloss:0.184139
[46]	eval-logloss:0.415564	train-logloss:0.183115
[47]	eval-logloss:0.416168	train-logloss:0.181754
[48]	eval-logloss:0.455506	train-logloss:0.179607
[49]	eval-logloss:0.464227	train-logloss:0.177548
[

[95]	eval-logloss:0.828814	train-logloss:0.10748
[96]	eval-logloss:0.829045	train-logloss:0.106385
[97]	eval-logloss:0.826059	train-logloss:0.105517
[98]	eval-logloss:0.86284	train-logloss:0.103794
[99]	eval-logloss:0.882346	train-logloss:0.102955
[0]	eval-logloss:0.626463	train-logloss:0.556883
[1]	eval-logloss:0.587109	train-logloss:0.47995
[2]	eval-logloss:0.587321	train-logloss:0.43134
[3]	eval-logloss:0.550687	train-logloss:0.398254
[4]	eval-logloss:0.534018	train-logloss:0.374685
[5]	eval-logloss:0.555858	train-logloss:0.35734
[6]	eval-logloss:0.538929	train-logloss:0.340615
[7]	eval-logloss:0.550351	train-logloss:0.327197
[8]	eval-logloss:0.580316	train-logloss:0.317374
[9]	eval-logloss:0.585881	train-logloss:0.308606
[10]	eval-logloss:0.560863	train-logloss:0.302296
[11]	eval-logloss:0.572873	train-logloss:0.29273
[12]	eval-logloss:0.554206	train-logloss:0.285639
[13]	eval-logloss:0.56598	train-logloss:0.279247
[14]	eval-logloss:0.533283	train-logloss:0.275291
[15]	eval-logloss

[61]	eval-logloss:0.549003	train-logloss:0.141421
[62]	eval-logloss:0.549419	train-logloss:0.140798
[63]	eval-logloss:0.548293	train-logloss:0.139756
[64]	eval-logloss:0.548165	train-logloss:0.139392
[65]	eval-logloss:0.546288	train-logloss:0.138933
[66]	eval-logloss:0.544504	train-logloss:0.137041
[67]	eval-logloss:0.531509	train-logloss:0.136341
[68]	eval-logloss:0.531832	train-logloss:0.135135
[69]	eval-logloss:0.529056	train-logloss:0.134724
[70]	eval-logloss:0.528856	train-logloss:0.13303
[71]	eval-logloss:0.552097	train-logloss:0.131741
[72]	eval-logloss:0.583087	train-logloss:0.129643
[73]	eval-logloss:0.600871	train-logloss:0.128721
[74]	eval-logloss:0.621433	train-logloss:0.127614
[75]	eval-logloss:0.59861	train-logloss:0.126151
[76]	eval-logloss:0.599922	train-logloss:0.125653
[77]	eval-logloss:0.601121	train-logloss:0.124863
[78]	eval-logloss:0.607575	train-logloss:0.123471
[79]	eval-logloss:0.610357	train-logloss:0.122926
[80]	eval-logloss:0.632237	train-logloss:0.122228
[8

...

[61]	eval-logloss:0.755228	train-logloss:0.145186
[62]	eval-logloss:0.756071	train-logloss:0.144603
[63]	eval-logloss:0.748901	train-logloss:0.14323
[64]	eval-logloss:0.756219	train-logloss:0.142381
[65]	eval-logloss:0.759067	train-logloss:0.139907
[66]	eval-logloss:0.787201	train-logloss:0.138736
[67]	eval-logloss:0.799046	train-logloss:0.137653
[68]	eval-logloss:0.813711	train-logloss:0.136099
[69]	eval-logloss:0.816364	train-logloss:0.134007
[70]	eval-logloss:0.837906	train-logloss:0.133391
[71]	eval-logloss:0.837891	train-logloss:0.131565
[72]	eval-logloss:0.838359	train-logloss:0.1295
[73]	eval-logloss:0.818929	train-logloss:0.128441
[74]	eval-logloss:0.845804	train-logloss:0.126436
[75]	eval-logloss:0.845859	train-logloss:0.124771
[76]	eval-logloss:0.84215	train-logloss:0.124415
[77]	eval-logloss:0.84193	train-logloss:0.123953
[78]	eval-logloss:0.853608	train-logloss:0.122088
[79]	eval-logloss:0.853324	train-logloss:0.121144
[80]	eval-logloss:0.852629	train-logloss:0.119697
[81]	

[26]	eval-logloss:0.707182	train-logloss:0.216185
[27]	eval-logloss:0.705634	train-logloss:0.212877
[28]	eval-logloss:0.683138	train-logloss:0.21123
[29]	eval-logloss:0.710335	train-logloss:0.208075
[30]	eval-logloss:0.720321	train-logloss:0.2047
[31]	eval-logloss:0.709522	train-logloss:0.203126
[32]	eval-logloss:0.709077	train-logloss:0.200328
[33]	eval-logloss:0.70326	train-logloss:0.197085
[34]	eval-logloss:0.710708	train-logloss:0.194408
[35]	eval-logloss:0.698443	train-logloss:0.190878
[36]	eval-logloss:0.703429	train-logloss:0.190201
[37]	eval-logloss:0.721471	train-logloss:0.18674
[38]	eval-logloss:0.732931	train-logloss:0.184154
[39]	eval-logloss:0.756988	train-logloss:0.182303
[40]	eval-logloss:0.783137	train-logloss:0.181122
[41]	eval-logloss:0.779621	train-logloss:0.179792
[42]	eval-logloss:0.779925	train-logloss:0.177208
[43]	eval-logloss:0.789474	train-logloss:0.173907
[44]	eval-logloss:0.754276	train-logloss:0.171047
[45]	eval-logloss:0.787204	train-logloss:0.167717
[46]	

[91]	eval-logloss:0.653045	train-logloss:0.099352
[92]	eval-logloss:0.634565	train-logloss:0.097781
[93]	eval-logloss:0.640192	train-logloss:0.096936
[94]	eval-logloss:0.639726	train-logloss:0.096189
[95]	eval-logloss:0.641408	train-logloss:0.094532
[96]	eval-logloss:0.654387	train-logloss:0.093156
[97]	eval-logloss:0.66035	train-logloss:0.09271
[98]	eval-logloss:0.688857	train-logloss:0.091306
[99]	eval-logloss:0.690181	train-logloss:0.090515
[0]	eval-logloss:0.600702	train-logloss:0.555782
[1]	eval-logloss:0.531099	train-logloss:0.480256
[2]	eval-logloss:0.483655	train-logloss:0.43318
[3]	eval-logloss:0.475076	train-logloss:0.396462
[4]	eval-logloss:0.459411	train-logloss:0.372562
[5]	eval-logloss:0.432775	train-logloss:0.352031
[6]	eval-logloss:0.456086	train-logloss:0.337649
[7]	eval-logloss:0.440542	train-logloss:0.325745
[8]	eval-logloss:0.419562	train-logloss:0.314577
[9]	eval-logloss:0.443512	train-logloss:0.302632
[10]	eval-logloss:0.44204	train-logloss:0.294545
[11]	eval-logl

[56]	eval-logloss:0.836468	train-logloss:0.155978
[57]	eval-logloss:0.836432	train-logloss:0.155219
[58]	eval-logloss:0.844212	train-logloss:0.1532
[59]	eval-logloss:0.848425	train-logloss:0.151184
[60]	eval-logloss:0.853012	train-logloss:0.15006
[61]	eval-logloss:0.857946	train-logloss:0.148257
[62]	eval-logloss:0.863909	train-logloss:0.146107
[63]	eval-logloss:0.866953	train-logloss:0.143977
[64]	eval-logloss:0.885501	train-logloss:0.141975
[65]	eval-logloss:0.868547	train-logloss:0.139481
[66]	eval-logloss:0.862775	train-logloss:0.137862
[67]	eval-logloss:0.870815	train-logloss:0.136685
[68]	eval-logloss:0.892163	train-logloss:0.136062
[69]	eval-logloss:0.887086	train-logloss:0.135418
[70]	eval-logloss:0.898644	train-logloss:0.133246
[71]	eval-logloss:0.860552	train-logloss:0.131588
[72]	eval-logloss:0.866895	train-logloss:0.130629
[73]	eval-logloss:0.867019	train-logloss:0.129846
[74]	eval-logloss:0.8635	train-logloss:0.127986
[75]	eval-logloss:0.868169	train-logloss:0.126539
[76]	

[21]	eval-logloss:0.495433	train-logloss:0.237463
[22]	eval-logloss:0.509183	train-logloss:0.234108
[23]	eval-logloss:0.496972	train-logloss:0.231179
[24]	eval-logloss:0.497868	train-logloss:0.226414
[25]	eval-logloss:0.507335	train-logloss:0.223364
[26]	eval-logloss:0.54879	train-logloss:0.220889
[27]	eval-logloss:0.581601	train-logloss:0.214669
[28]	eval-logloss:0.584839	train-logloss:0.211225
[29]	eval-logloss:0.566697	train-logloss:0.20923
[30]	eval-logloss:0.552787	train-logloss:0.207157
[31]	eval-logloss:0.542488	train-logloss:0.205355
[32]	eval-logloss:0.560042	train-logloss:0.202611
[33]	eval-logloss:0.543159	train-logloss:0.199709
[34]	eval-logloss:0.574097	train-logloss:0.194617
[35]	eval-logloss:0.563526	train-logloss:0.191456
[36]	eval-logloss:0.55563	train-logloss:0.189129
[37]	eval-logloss:0.556735	train-logloss:0.188048
[38]	eval-logloss:0.561277	train-logloss:0.183963
[39]	eval-logloss:0.561289	train-logloss:0.182306
[40]	eval-logloss:0.536327	train-logloss:0.180823
[41

In [69]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [70]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [71]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [72]:
# Hidden Test Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [73]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [74]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [75]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [76]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [77]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [78]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [79]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [80]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [81]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [82]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#
